In [1]:

files = [("../sodata/tensorflow/code.txt", "../sodata/tensorflow/title.txt", "../sodata/tensorflow/meta.txt"),
         ("../sodata/machine-learning_python/code.txt","../sodata/machine-learning_python/title.txt","../sodata/machine-learning_python/meta.txt"),
         ("../sodata/data-science_python/code.txt","../sodata/data-science_python/title.txt","../sodata/data-science_python/meta.txt"),
         ("../sodata/data-cleaning_python/code.txt","../sodata/data-cleaning_python/title.txt","../sodata/data-cleaning_python/meta.txt"),
         ("../sodata/data-science_python/code.txt","../sodata/data-science_python/title.txt","../sodata/data-science_python/meta.txt")]

In [2]:
def process_code_file(path):
    file = open(path, 'r', encoding="utf-8")
    code_file = file.read()

    processed_code = []

    for entry in code_file.split("\n"):

        filtered_lanes = ""

        for line in entry.split("\\n"):
            line = line.strip()
            line = line.replace("(", " ").replace(")", " ").replace("\"", " ").replace("'", " ").replace("&quot;", " ").replace("_", " ").replace(".", " ").replace(",", " ").replace("=", " ")
            line = line.replace(":", " ").replace("[", " ").replace("]", " ").replace("\\"," ").replace("/"," ")
            line = line.replace("+"," ").replace("-"," ").replace("_"," ").replace("&gt;", " ").replace("{","").replace("}","")
            line = line.replace("%", " ").replace("$", " ")
            # remove numbers
            line = res = re.sub('([0-9]+.[0-9]+|[0-9]+)', ' NUM ', line) 
            
            # Split camelcase
            line = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', line))
            
            line = line.lower()
            line = line.strip()
            if line.startswith("print") or line.startswith("#") or line.startswith("\"#") or line == "" or line == "\"" :
                continue
            
            # remove non alphanumeric characters
            line = re.sub("[^0-9a-zA-Z ]+", ' ', line)
            
            filtered_lanes += line+" "

        processed_code.append(filtered_lanes)

    file.close()
    return processed_code  

In [3]:
def process_title_file(path):
    file = open(path, 'r', encoding="utf-8")
    tile_file = file.read()
    tile_file = tile_file.replace("?", " ").replace(",", " ").replace(".", " ").replace(")", " ").replace("(", " ")
    tile_file = tile_file.replace(":", " ").replace("`", "").replace("[", " ").replace("]", " ")
    tile_file = tile_file.split("\n")
    return tile_file

In [4]:
def process_meta_file(path):
    file = open(path, 'r', encoding="utf-8")
    meta_file = file.read()
    return meta_file.split("\n")

In [13]:
import random
import re

codes = []
titles = []
urls = []
for file in files:
    codes.extend(process_code_file(file[0]))
    titles.extend(process_title_file(file[1]))
    urls.extend(process_meta_file(file[2]))
    
c = list(zip(codes, titles, urls))

random.shuffle(c)

codes, titles, urls = zip(*c)

codes = list(codes)
titles = list(titles)
urls = list(urls)

In [14]:
#codes[0]

urls[0]

'46995313,https://stackoverflow.com/questions/46995313/list-position-replacement-with-if-statement'

In [7]:
test_division_pos =  int(len(codes) * 0.8)

training_codes = codes[:test_division_pos]
training_titles = titles[:test_division_pos]

test_codes = codes[test_division_pos:]
test_titles = titles[test_division_pos:]

print(len(training_codes), len(test_codes))

16809 4203


In [8]:
import tensorflow as tf

def get_tokenizer(tokens):
    top_k = 5000000
    tokenizer_code = tf.keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                      oov_token="<unk>",
                                                      filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')

    tokenizer_code.word_index['<pad>'] = 0
    tokenizer_code.index_word[0] = '<pad>'
    tokenizer_code.fit_on_texts(tokens)
    
    return tokenizer_code
#number_desc_tokens = len(tokenizer_code.word_index) + 1
#print(number_desc_tokens)

In [9]:
from bpe import Encoder

def get_bytepair_tokenizer():
    test_corpus = '''
        Object raspberrypi functools dict kwargs. Gevent raspberrypi functools. Dunder raspberrypi decorator dict didn't lambda zip import pyramid, she lambda iterate?
        Kwargs raspberrypi diversity unit object gevent. Import fall integration decorator unit django yield functools twisted. Dunder integration decorator he she future. Python raspberrypi community pypy. Kwargs integration beautiful test reduce gil python closure. Gevent he integration generator fall test kwargs raise didn't visor he itertools...
        Reduce integration coroutine bdfl he python. Cython didn't integration while beautiful list python didn't nit!
        Object fall diversity 2to3 dunder script. Python fall for: integration exception dict kwargs dunder pycon. Import raspberrypi beautiful test import six web. Future integration mercurial self script web. Return raspberrypi community test she stable.
        Django raspberrypi mercurial unit import yield raspberrypi visual rocksdahouse. Dunder raspberrypi mercurial list reduce class test scipy helmet zip?
    '''

    encoder = Encoder(200, pct_bpe=0.88)  # params chosen for demonstration purposes
    encoder.fit(test_corpus.split('\n'))

    example = "Vizzini: He didn't fall? INCONCEIVABLE!"
    print(encoder.tokenize(example))
    # ['__sow', 'vi', 'z', 'zi', 'ni', '__eow', '__sow', ':', '__eow', 'he', 'didn', "'", 't', 'fall', '__sow', '?', '__eow', '__sow', 'in', 'co', 'n', 'ce', 'iv', 'ab', 'le', '__eow', '__sow', '!', '__eow']
    print(next(encoder.transform([example])))
    # [24, 108, 82, 83, 71, 25, 24, 154, 25, 14, 10, 11, 12, 13, 24, 85, 25, 24, 140, 59, 39, 157, 87, 165, 114, 25, 24, 148, 25]
    print(next(encoder.inverse_transform(encoder.transform([example]))))

In [10]:
code_train_tokenizer = get_tokenizer(training_codes)
code_test_tokenizer = get_tokenizer(test_codes)

In [11]:
print( len(code_train_tokenizer.word_index) + 1)
print( len(code_test_tokenizer.word_index) + 1)

15354
8211


In [12]:
included = 0
for test_word in code_test_tokenizer.word_index:
    if test_word in code_train_tokenizer.word_index:
        included += 1
        
print("% of test words are in the training set", included/len(code_test_tokenizer.word_index))

% of test words are in the training set 0.7506699147381243


In [13]:
title_train_tokenizer = get_tokenizer(training_titles)
title_test_tokenizer = get_tokenizer(test_titles)

In [14]:
print( len(title_train_tokenizer.word_index) + 1)
print( len(title_test_tokenizer.word_index) + 1)

7329
3881


In [15]:
included = 0
for test_word in title_test_tokenizer.word_index:
    if test_word in title_train_tokenizer.word_index:
        included += 1
        
print("% of test words are in the training set", included/len(title_test_tokenizer.word_index))

% of test words are in the training set 0.8260309278350515


In [16]:
# Mixed vocab
extended_training = []
extended_test = []
extended_training.extend(training_codes)
extended_training.extend(training_titles)
extended_test.extend(test_codes)
extended_test.extend(test_titles)

extended_train_tokenizer = get_tokenizer(extended_training)
extended_test_tokenizer = get_tokenizer(extended_test)

print( len(extended_train_tokenizer.word_index) + 1)
print( len(extended_test_tokenizer.word_index) + 1)

included = 0
for test_word in extended_test_tokenizer.word_index:
    if test_word in extended_train_tokenizer.word_index:
        included += 1
        
print("% of test words are in the training set", included/len(extended_test_tokenizer.word_index))

18163
9653
% of test words are in the training set 0.7588064649813511


In [17]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

In [18]:
tokenizer = Tokenizer(BPE(vocab_size=40000, unk_token="[UNK]"))
tokenizer.pre_tokenizer = Whitespace(vocab_size=40000)

trainer = BpeTrainer(vocab_size=40000, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]) 

tokenizer.train_from_iterator(training_codes, trainer=trainer)

In [19]:
included = 0
for test_word in code_test_tokenizer.word_index:
    output = tokenizer.encode(test_word, add_special_tokens=True)
    if not test_word in code_train_tokenizer.word_index:
        print("Problematic word", test_word, output.tokens)
        if "[UNK]" in output.tokens:
            print("Word not found", test_word, output.tokens)
        else:
            included += 1  
    else:
        #print("Good word", test_word, output.tokens)
        included += 1

print("% of test words are in the training set", included/len(code_test_tokenizer.word_index))

Problematic word graduate ['grad', 'u', 'ate']
Problematic word kkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkk ['kkkk', 'kkkk', 'kkkk', 'kkkk', 'kkkk', 'kkkk', 'kkkk', 'kkkk', 'kkkk', 'kkkk', 'kkkk', 'kkkk', 'kkkk', 'kkkk', 'kkkk', 'kkkk']
Problematic word astronauts ['as', 'tr', 'on', 'aut', 's']
Problematic word btc ['bt', 'c']
Problematic word rotating ['rot', 'ating']
Problematic word death ['death']
Problematic word astr ['as', 'tr']
Problematic word ineq ['ine', 'q']
Problematic word datim ['dat', 'im']
Problematic word generosity ['gener', 'os', 'ity']
Problematic word trust ['trust']
Problematic word netout ['net', 'out']
Problematic word spacex ['space', 'x']
Problematic word tds ['t', 'ds']
Problematic word maksym ['ma', 'k', 'sym']
Problematic word economy ['e', 'con', 'o', 'my']
Problematic word milka ['mil', 'ka']
Problematic word textacy ['text', 'acy']
Problematic word clonx ['clon', 'x']
Problematic word insulin ['ins', 'ul', 'in']
Problematic word sds

Problematic word peach ['peach']
Problematic word neighbour ['neighbour']
Problematic word ivh ['iv', 'h']
Problematic word grap ['gra', 'p']
Problematic word bark ['bar', 'k']
Problematic word eastern ['e', 'aster', 'n']
Problematic word inactivity ['in', 'activity']
Problematic word thezip ['the', 'zip']
Problematic word actu ['act', 'u']
Problematic word nnz ['nn', 'z']
Problematic word argmaxes ['argmax', 'es']
Problematic word boolmask ['bool', 'mask']
Problematic word listdiff ['list', 'diff']
Problematic word dither ['d', 'ith', 'er']
Problematic word sweep ['swe', 'ep']
Problematic word linecolor ['line', 'color']
Problematic word turtles ['tur', 't', 'les']
Problematic word ofile ['o', 'file']
Problematic word drove ['dro', 've']
Problematic word dorsal ['d', 'ors', 'al']
Problematic word medial ['medi', 'al']
Problematic word plantar ['plan', 'tar']
Problematic word lateral ['later', 'al']
Problematic word distal ['dist', 'al']
Problematic word initia ['init', 'ia']
Problemat

Problematic word reuses ['re', 'uses']
Problematic word digs ['di', 'gs']
Problematic word capgen ['cap', 'gen']
Problematic word moderator ['mod', 'er', 'ator']
Problematic word incertezza ['inc', 'ert', 'e', 'zz', 'a']
Problematic word togliere ['to', 'g', 'li', 'ere']
Problematic word multicol ['multi', 'col']
Problematic word gath ['gath']
Problematic word sequneces ['seq', 'une', 'ces']
Problematic word repacking ['re', 'pack', 'ing']
Problematic word zerod ['zer', 'od']
Problematic word cmsis ['c', 'ms', 'is']
Problematic word bsp ['b', 'sp']
Problematic word immediate ['im', 'medi', 'ate']
Problematic word jemalloc ['je', 'malloc']
Problematic word puntuacion ['p', 'unt', 'u', 'ac', 'ion']
Problematic word pddata ['pd', 'data']
Problematic word sexid ['sex', 'id']
Problematic word edgeitems ['edge', 'items']
Problematic word rnnx ['rnn', 'x']
Problematic word minimalrnn ['minimal', 'rnn']
Problematic word sigmoide ['sig', 'mo', 'ide']
Problematic word rnds ['rn', 'ds']
Problemat

Problematic word cosines ['cos', 'ines']
Problematic word redirector ['redirect', 'or']
Problematic word detoxing ['de', 'to', 'x', 'ing']
Problematic word lexapro ['le', 'xa', 'pro']
Problematic word ipv ['ip', 'v']
Problematic word xxxxx ['xxxx', 'x']
Problematic word umit ['um', 'it']
Problematic word checkpointed ['checkpoint', 'ed']
Problematic word lrv ['lr', 'v']
Problematic word confint ['conf', 'int']
Problematic word didnt ['didn', 't']
Problematic word btw ['b', 'tw']
Problematic word workflow ['work', 'flow']
Problematic word chardet ['char', 'det']
Problematic word radam ['ra', 'dam']
Problematic word nba ['nb', 'a']
Problematic word touts ['tout', 's']
Problematic word solubility ['sol', 'ub', 'ility']
Problematic word deallocated ['de', 'allocated']
Problematic word subsequent ['sub', 'sequ', 'ent']
Problematic word cycling ['cy', 'cl', 'ing']
Problematic word mydirectory ['my', 'directory']
Problematic word reordered ['re', 'ordered']
Problematic word docuemnt ['doc', '

Problematic word kkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkk ['kkkk', 'kkkk', 'kkkk', 'kkkk', 'kkkk', 'kkkk', 'kkkk', 'kkkk']
Problematic word istate ['i', 'state']
Problematic word lenth ['len', 'th']
Problematic word forecasts ['forecast', 's']
Problematic word outliner ['out', 'lin', 'er']
Problematic word abnormal ['ab', 'normal']
Problematic word parsefunc ['parse', 'func']
Problematic word mygreater ['my', 'greater']
Problematic word myless ['my', 'less']
Problematic word nnnn ['nn', 'nn']
Problematic word ipkg ['ip', 'kg']
Problematic word rfv ['rf', 'v']
Problematic word thetay ['theta', 'y']
Problematic word lightgreen ['light', 'green']
Problematic word yellowgreen ['yellow', 'green']
Problematic word repl ['re', 'pl']
Problematic word hopefully ['hope', 'fully']
Problematic word revoke ['re', 'vo', 'ke']
Problematic word warmstarting ['warm', 'starting']
Problematic word rsample ['r', 'sample']
Problematic word pn ['pn']
Problematic word armadillo ['arma', 'di', 'l', 'lo']
Problematic 

Problematic word frequently ['frequent', 'ly']
Problematic word speak ['spe', 'ak']
Problematic word forceful ['force', 'ful']
Problematic word monologue ['mono', 'log', 'ue']
Problematic word crisp ['cr', 'is', 'p']
Problematic word engross ['en', 'gross']
Problematic word subtitle ['sub', 'title']
Problematic word knowledge ['knowledge']
Problematic word lapp ['lap', 'p']
Problematic word era ['era']
Problematic word dramatic ['dra', 'matic']
Problematic word transcend ['tran', 'sc', 'end']
Problematic word kudos ['k', 'udo', 's']
Problematic word talented ['tal', 'ent', 'ed']
Problematic word cinematographer ['c', 'ine', 'mato', 'graph', 'er']
Problematic word obscurity ['ob', 'sc', 'urity']
Problematic word ripe ['ri', 'pe']
Problematic word rediscovery ['re', 'discovery']
Problematic word wonder ['wonder']
Problematic word woody ['wood', 'y']
Problematic word wannabe ['wanna', 'be']
Problematic word imaginatively ['ima', 'gin', 'ativ', 'ely']
Problematic word enjoyable ['enjoy', '

Problematic word aquarium ['a', 'qu', 'ar', 'ium']
Problematic word beaver ['be', 'aver']
Problematic word bee ['be', 'e']
Problematic word beetle ['be', 'et', 'le']
Problematic word butterfly ['butter', 'f', 'ly']
Problematic word castle ['cast', 'le']
Problematic word caterpillar ['cat', 'er', 'pillar']
Problematic word cattle ['cat', 'tle']
Problematic word chimpanzee ['ch', 'imp', 'an', 'ze', 'e']
Problematic word cockroach ['co', 'ck', 'ro', 'ach']
Problematic word crocodile ['cro', 'cod', 'ile']
Problematic word flatfish ['flat', 'fish']
Problematic word lawn ['la', 'wn']
Problematic word mower ['mo', 'wer']
Problematic word leopard ['leop', 'ard']
Problematic word lizard ['l', 'iz', 'ard']
Problematic word maple ['map', 'le']
Problematic word mountain ['mo', 'un', 'tain']
Problematic word mushroom ['mush', 'room']
Problematic word oak ['o', 'ak']
Problematic word orchid ['orch', 'id']
Problematic word otter ['ot', 'ter']
Problematic word poppy ['pop', 'py']
Problematic word porc

In [20]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

tokenizer = Tokenizer(BPE(vocab_size=40000, unk_token="[UNK]"))
tokenizer.pre_tokenizer = Whitespace(vocab_size=40000)

trainer = BpeTrainer(vocab_size=40000, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]) 

# training_codes
# training_titles
tokenizer.train_from_iterator(training_codes, trainer=trainer)

len(tokenizer.get_vocab())

19614

In [21]:
tokenizer = Tokenizer(BPE(vocab_size=40000, unk_token="[UNK]"))
tokenizer.pre_tokenizer = Whitespace(vocab_size=40000)

trainer = BpeTrainer(vocab_size=40000, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]) 

# training_codes
# training_titles
tokenizer.train_from_iterator(training_titles, trainer=trainer)

len(tokenizer.get_vocab())

15844